# Insert

As the keyword implies, INSERT adds records to a table.

However, as we tend to find with the SQL language, there are many different syntaxes we can use around the keyword to achieve our aims so we will cover them all here, starting with the most basic.

However, firstly, we need to create a temporary table into which we can add some data. To follow along from our Gym Member Table that we looked at in the Table Creation section we shall start as follows:

Our table will be relatively basic, only catering for:

-   memberNumber (integer)
-   firstName (string of up to 25 characters)
-   lastName (string of up to 35 characters)
-   fullName (string of up to 60 characters)
-   isActive (bit column)

Note - We could utilise a permanent table for these demos, but using a temporary table ensures that multiple users can run these demos at the same time without issue. A permanent user table would cause issues in this regard. Hence we will stick to the temporary table.  We must also remember to drop these before trying to use them (using the SQL Server 2016 and above method for these demos).

#### Basic Insert

In the most basic form we simply state that we wish Insert into a table and we present it with data we know will fit.

We use the syntax:

`INSERT INTO _tableName_`

And we pass in data using SELECT:

In [ ]:
drop table if exists #memberTable;

create table #memberTable
(
    memberNumber int,
    firstName varchar(25),
    lastName varchar(35),
    fullName varchar(60),
    isActive bit
);

insert into #memberTable
select 1, 'John', 'Smith', 'John Smith', 1;

select *
from #memberTable;

Obviously this method is prone to errors with the risk of providing an incorrect number of columns to the table or passing data in the wrong order.

Therefore a more correct approach is to use a column list to control the entry of our data into our table:

In [ ]:
drop table if exists #memberTable;

create table #memberTable
(
    memberNumber int,
    firstName varchar(25),
    lastName varchar(35),
    fullName varchar(60),
    isActive bit
);

insert into #memberTable
(
    memberNumber, firstName, lastName, fullName, isActive
)
select 1, 'John', 'Smith', 'John Smith', 1;

select *
from #memberTable;

We can also use this method to enter data in a different order to the table itself by mapping it through the column list.

This can sometimes be essential if the data being imported is coming from a third party and in a set order yet our table is ordered in a different manner.

In the following example our data has 'arrived' with last name and first name in the opposite order to the table, therefore we can use a column list to accommodate and map this:

In [ ]:
drop table if exists #memberTable;

create table #memberTable
(
    memberNumber int,
    firstName varchar(25),
    lastName varchar(35),
    fullName varchar(60),
    isActive bit
);

insert into #memberTable
(
    memberNumber, lastName, firstName, fullName, isActive
)
select 1, 'Smith', 'John', 'John Smith', 1;

select *
from #memberTable;

This is a basic INSERT using hand written and hard coded values.

We can also INSERT our data by selecting it from another table and using that as our source.

#### Insert From Another Table

Most often we will be importing data from one table, manipulating the data (ie. taking a subset of records or columns and potentially also transforming the data as well), and inserting into our new table.

This is perfectly supported within SQL Server and, to demonstrate, we will use our old friend the hr.employee table.

Let's write a query from hr.employee, returning the firstName, lastName, and fullName (manually calculated) for the first 5 people in our table (employeeID 1 to 5):

In [ ]:
select firstName, lastName, CONCAT_WS(' ', firstName, lastName)
from JupyterDatabase.hr.employee
where employeeID <= 5;

We now need a memberNumber (we could use the employeeID but, in order to practice some other techniques, I wish to presume this column doesn't exist on our table and therefore we need to manually append an identity value), and we also need to flag all our new members as being 'active':

In [ ]:
select row_number() over(order by firstName) as memberNumber,
        firstName, lastName, CONCAT_WS(' ', firstName, lastName) as fullName,
        1 as isActive
from JupyterDatabase.hr.employee
where employeeID <= 5;

Great, we now have our data, therefore we can now INSERT it into our temporary table.

The syntax for this is exactly the same as with the INSERT we did above, except that we are generating our data from a table rather than typing it manually:

In [ ]:
drop table if exists #memberTable;

create table #memberTable
(
    memberNumber int,
    firstName varchar(25),
    lastName varchar(35),
    fullName varchar(60),
    isActive bit
);

insert into #memberTable
(
    memberNumber, firstName, lastName, fullName, isActive
)
select row_number() over(order by firstName) as memberNumber,
        firstName, lastName, CONCAT_WS(' ', firstName, lastName) as fullName,
        1 as isActive
from JupyterDatabase.hr.employee
where employeeID <= 5;

select *
from #memberTable;